In [0]:
%load_ext autoreload
%autoreload 2

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Hashing")

In [0]:
import warnings 
warnings.filterwarnings("ignore")

In [0]:
import os, sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks/text-generator")

In [0]:
from utils.data_loader import load_dataset

In [11]:
from utils.data_loader import load_test_dataset

root path: /content/gdrive/My Drive/Colab Notebooks


In [34]:
from utils.config import embedding_matrix_path
embedding_matrix_path

'/content/gdrive/My Drive/Colab Notebooks/Hashing/gen_data/embedding_matrix'

In [42]:
from utils.wv_loader import load_embedding_matrix
load_embedding_matrix()

array([[ 5.78204274e-01,  6.03120551e-02,  3.60607475e-01, ...,
        -3.83604765e-01, -2.35126719e-01, -3.39127541e-01],
       [ 2.48232469e-01, -1.09330297e-01, -1.07853085e-01, ...,
        -4.05686855e-01, -3.84583443e-01,  8.72177184e-01],
       [-2.61794060e-01,  1.58479378e-01, -3.25242400e-01, ...,
        -1.04319692e+00, -3.70189011e-01, -6.24906011e-02],
       ...,
       [ 3.13954473e-01,  6.27483010e-01,  8.70058371e-04, ...,
        -6.59883201e-01,  1.04633875e-01,  5.38641512e-02],
       [-3.03027540e-01, -6.11446559e-01, -1.08515084e+00, ...,
         1.52157664e-01,  3.57411057e-01, -8.81655991e-01],
       [ 3.11857313e-01, -5.35719246e-02, -3.20000172e-01, ...,
        -1.04822442e-01,  1.92798018e-01, -4.42412287e-01]], dtype=float32)

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from utils.data_loader import load_dataset, preprocess_sentence, load_test_dataset
from utils.wv_loader import load_embedding_matrix, load_vocab
from utils.config import *
from gensim.models.word2vec import LineSentence, Word2Vec
import tensorflow as tf
# from utils.plot_utils import plot_attention
from tqdm import tqdm
import time
from seq2seq.batcher import train_batch_generator

In [0]:
vocab_path = "./gen_data/vocab.txt"
vocab, reverse_vocab=load_vocab(vocab_path)

In [0]:
train_X, train_Y, test_X = load_dataset()

In [21]:
from utils.wv_loader import get_embedding_matrix
wv_model_path = "./gen_data/word2vec.model"
embedding_matrix = get_embedding_matrix(wv_model_path)

2020-05-18 14:03:11,689 : INFO : loading Word2Vec object from ./gen_data/word2vec.model
2020-05-18 14:03:13,649 : INFO : loading wv recursively from ./gen_data/word2vec.model.wv.* with mmap=None
2020-05-18 14:03:13,650 : INFO : setting ignored attribute vectors_norm to None
2020-05-18 14:03:13,651 : INFO : loading vocabulary recursively from ./gen_data/word2vec.model.vocabulary.* with mmap=None
2020-05-18 14:03:13,656 : INFO : loading trainables recursively from ./gen_data/word2vec.model.trainables.* with mmap=None
2020-05-18 14:03:13,658 : INFO : setting ignored attribute cum_table to None
2020-05-18 14:03:13,659 : INFO : loaded ./gen_data/word2vec.model


In [0]:
np.save("./gen_data/embedding_matrix.npy", embedding_matrix)

In [0]:
units = 1024

In [0]:
params = {}
params["vocab_size"] = len(vocab)
params["embed_size"] = 300
params["enc_units"] = units
params["attn_units"] = units
params["dec_units"] = units
params["batch_size"] = 64
params["epochs"] = 5
params["max_enc_len"] = 200
params["max_dec_len"] = 41

In [0]:
steps_per_epoch = len(train_X) // params["batch_size"]

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(params["batch_size"])
dataset = dataset.batch(params["batch_size"], drop_remainder=True)

In [0]:
from seq2seq.layers import Encoder, BahdanauAttention, Decoder

In [0]:
encoder = Encoder(params["vocab_size"], params["embed_size"], embedding_matrix, params["enc_units"], params["batch_size"])

In [0]:
enc_hidden = encoder.initialize_hidden_state()

In [0]:
example_input_batch = tf.ones(shape=(params["batch_size"], params["max_enc_len"]), dtype=tf.int32)

In [82]:
sample_output, sample_hidden = encoder(example_input_batch, enc_hidden)
sample_output.shape
sample_hidden.shape

(64, 200, 300)


TensorShape([64, 1024])

In [0]:
decoder = Decoder(params["vocab_size"], params["embed_size"], embedding_matrix, params["enc_units"], params["batch_size"])

In [0]:
sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)

In [86]:
sample_decoder_output.shape

TensorShape([64, 32909])

In [0]:
# from seq2seq.models import Seq2Seq

In [0]:
# model = Seq2Seq(params)

In [0]:
checkpoint_dir = "./gen_data/checkpoints/training_checkpoints_mask_loss_dim300_seq"
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

In [0]:
ckpt = tf.train.Checkpoint(Seq2Seq=model)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)

In [0]:
## 训练

In [0]:
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


pad_index=vocab['<PAD>']
nuk_index=vocab['<UNK>']

def loss_function(real, pred):
    pad_mask = tf.math.equal(real, pad_index)
    nuk_mask = tf.math.equal(real, nuk_index)
    mask = tf.math.logical_not(tf.math.logical_or(pad_mask,nuk_mask))
    
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        # 1. 构建encoder
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        # 2. 复制
        dec_hidden = enc_hidden
        # 3. <START> * BATCH_SIZE 
        dec_input = tf.expand_dims([vocab['<START>']] * params["batch_size"], 1)
        
        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # decoder(x, hidden, enc_output)
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            
            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))

        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [0]:
epochs = params["epochs"]
# 如果检查点存在，则恢复最新的检查点。
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')
    
for epoch in range(epochs):
    start = time.time()
    total_loss = 0
    enc_hidden = encoder.initialize_hidden_state()

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 1 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             ckpt_save_path))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

(64, 260, 300)
(64, 260, 300)
Epoch 1 Batch 0 Loss 5.7895
Epoch 1 Batch 1 Loss 5.0485
Epoch 1 Batch 2 Loss 4.6711
Epoch 1 Batch 3 Loss 4.4872
Epoch 1 Batch 4 Loss 2.9991
Epoch 1 Batch 5 Loss 2.8944
Epoch 1 Batch 6 Loss 2.2248
Epoch 1 Batch 7 Loss 2.3178
Epoch 1 Batch 8 Loss 2.5467
Epoch 1 Batch 9 Loss 3.2641
Epoch 1 Batch 10 Loss 3.4364
Epoch 1 Batch 11 Loss 2.9485
Epoch 1 Batch 12 Loss 2.7538
